<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# 자동 생성된 상담원 채팅: 코딩 및 계획 에이전트를 통한 협업 작업 해결

자동 생성은 자동화된 채팅을 통해 공동으로 작업을 수행하는 데 사용할 수 있는 LLM, 도구 또는 사람으로 구동되는 대화형 에이전트를 제공합니다. 이 프레임워크는 여러 에이전트 간의 대화를 통해 툴 사용과 사람 참여가 가능합니다.
이 기능에 대한 설명서는 [여기](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)에서 확인할 수 있습니다.

이 노트북에서는 여러 에이전트를 사용하여 함께 작업하고 웹에서 정보를 찾고 코딩해야 하는 작업을 수행하는 방법을 보여드립니다. ASSISTANT 에이전트`는 사용자가 주어진 작업에 대해 실행할 Python 코드를 작성하고 디버그할 수 있는 LLM 기반 에이전트입니다(Python 코딩 블록에서). UserProxyAgent`는 사용자가 `AssistantAgent`가 작성한 코드를 실행할 수 있도록 프록시 역할을 하는 에이전트입니다. 보조 에이전트가 참조할 계획 에이전트를 추가로 생성합니다. 플래닝 에이전트는 시스템 메시지가 다른 LLM 기반 `AssistantAgent`의 변형입니다.

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 파이오토젠과 도커를 설치하세요:
```bash
pip 설치 파이오토젠 도커
```

In [1]:
# %pip install pyautogen~=0.1.0 docker

## API 엔드포인트 설정

 이 함수는 API 키와 API 베이스가 해당 환경 변수 또는 로컬 txt 파일에 저장되어 있다고 가정합니다:
  - OpenAI API 키: os.environ["OPENAI_API_KEY"] 또는 `openai_api_key_file="key_openai.txt"`.
  - Azure OpenAI API 키: os.environ["AZURE_OPENAI_API_KEY"] 또는 `aoai_api_key_file="key_aoai.txt"`. 한 줄에 하나씩 여러 개의 키를 저장할 수 있습니다.
  - Azure OpenAI API 베이스: os.environ["AZURE_OPENAI_API_BASE"] 또는 `aoai_api_base_file="base_aoai.txt"`. 한 줄에 하나씩 여러 개의 베이스를 저장할 수 있습니다.
* `config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다. 먼저 지정된 이름의 환경 변수를 찾습니다. 환경 변수의 값은 유효한 json 문자열이어야 합니다. 해당 변수를 찾을 수 없으면 같은 이름의 json 파일을 찾습니다. filter_dict를 기준으로 구성을 필터링합니다.

OpenAI API 키만 있거나, Azure OpenAI API 키 + 베이스만 있어도 괜찮습니다. 콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 "파일 업로드" 아이콘을 선택해 파일을 업로드할 수 있습니다.


In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

구성 목록은 다음과 같습니다:
```python
config_list = [
    {
        '모델': 'gpt-4',
        'api_key': '<여기에 OpenAI API 키 입력>',
    }, # gpt-4용 OpenAI API 엔드포인트
    {
        'model': 'gpt-4',
        'api_key': '<귀하의 Azure OpenAI API 키는 여기>',
        'api_base': '<귀하의 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-07-01-preview',
    }, # gpt-4용 Azure OpenAI API 엔드포인트
    {
        'model': 'gpt-4-32k',
        'api_key': '<귀하의 Azure OpenAI API 키는 여기>',
        'api_base': '<귀하의 Azure OpenAI API 기반은 여기>',
        'api_type': 'azure',
        'api_version': '2023-07-01-preview',
    }, # gpt-4-32k용 Azure OpenAI API 엔드포인트
]
```

콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 "파일 업로드" 아이콘을 선택하여 파일을 업로드할 수 있습니다.

YAML 파일에서 불러오기 등 다른 방법으로 config_list의 값을 설정할 수 있습니다.

## 에이전트 구성

"planner"라는 이름의 플래너 에이전트와 "planner_user"라는 플래너에 대한 사용자 프록시 에이전트를 구성합니다. 사용자 프록시 에이전트에서 `human_input_mode`를 "NEVER"로 지정하여 사람의 피드백을 요청하지 않도록 합니다. 플래너에게 메시지를 보내고 플래너로부터 제안을 반환하는 `ask_planner` 함수를 정의합니다.

In [3]:
planner = autogen.AssistantAgent(
    name="planner",
    llm_config={"config_list": config_list},
    # the default system message of the AssistantAgent is overwritten here
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step that can be implemented by writing code. For example, browsing the web can be implemented by writing code that reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix."
)
planner_user = autogen.UserProxyAgent(
    name="planner_user",
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
)

def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    # return the last message received from the planner
    return planner_user.last_message()["content"]


어시스턴트 에이전트와 사용자 프록시 에이전트를 구성합니다. 사용자 프록시 에이전트에서 `human_input_mode`를 "종료"로 지정하면 어시스턴트 에이전트로부터 "종료" 신호를 받으면 피드백을 요청합니다. 생성한 `ask_planner` 함수를 사용하도록 `AssistantAgent`의 `functions`와 `UserProxyAgent`의 `function_map`을 설정합니다.

In [4]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "temperature": 0,
        "request_timeout": 600,
        "seed": 42,
        "model": "gpt-4-0613",
        "config_list": autogen.config_list_openai_aoai(exclude="aoai"),
        "functions": [
            {
                "name": "ask_planner",
                "description": "ask planner to: 1. get a plan for finishing a task, 2. verify the execution result of the plan and potentially suggest new plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask planner. Make sure the question include enough context, such as the code and the execution result. The planner does not know the conversation between you and the user, unless you share the conversation with the planner.",
                        },
                    },
                    "required": ["message"],
                },
            },
        ],
    }
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "planning"},
    function_map={"ask_planner": ask_planner},
)

## 작업 수행

사용자 프록시 에이전트의 `initiate_chat()` 메서드를 호출하여 대화를 시작합니다. 아래 셀을 실행하면 어시스턴트 에이전트가 메시지 끝에 "종료" 신호를 보낸 후 피드백을 제공할지 묻는 메시지가 표시됩니다. 피드백을 제공하지 않으면(직접 Enter 키를 눌러) 대화가 종료됩니다. "종료" 신호가 오기 전에 사용자 프록시 에이전트는 사용자를 대신하여 보조 상담원이 제안한 코드를 실행하려고 시도합니다.

In [5]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Suggest a fix to an open good first issue of flaml""",
)

user_proxy (to assistant):

Suggest a fix to an open good first issue of flaml

--------------------------------------------------------------------------------
assistant (to user_proxy):

To suggest a fix to an open good first issue of FLAML, we first need to fetch the list of open issues labeled as "good first issue" from the FLAML GitHub repository. We can do this using the GitHub API.

Here is a Python script that uses the requests library to fetch the list of open issues labeled as "good first issue" from the FLAML GitHub repository.

```python
# filename: fetch_issues.py

import requests
import json

def fetch_issues():
    url = "https://api.github.com/repos/microsoft/FLAML/issues"
    params = {
        "state": "open",
        "labels": "good first issue"
    }
    response = requests.get(url, params=params)
    issues = response.json()
    for issue in issues:
        print(f"Issue ID: {issue['id']}, Title: {issue['title']}, URL: {issue['html_url']}")

fetch_issues()
```

Ple

어시스턴트가 플래너와 상담해야 할 때 `ask_planner`에 대한 함수 호출을 제안합니다. 이 경우 다음과 같은 줄이 표시됩니다:

***** 제안된 함수 호출: ask_planner *****
